In [1]:
HOPSWORKS_PROJECT_NAME = 'taxi_demand_service'

In [2]:
import os
from dotenv import load_dotenv
from src.paths import PARENT_DIR

# load key-value pairs from .env file located in the parent directory
load_dotenv(PARENT_DIR / '.env')

HOPSWORKS_API_KEY = os.environ['HOPSWORKS_API_KEY']


In [3]:
from datetime import datetime
import pandas as pd

from src.data import load_raw_data

from_year = 2023
to_year = datetime.now().year
print(f'Downloading raw data from {from_year} to {to_year}')

rides = pd.DataFrame()

for year in range(from_year, to_year + 1):
  # download data for the whole year
  rides_one_year = load_raw_data(year)

  # append rows
  rides = pd.concat([rides, rides_one_year])

File 2023-01 was already in local storage
File 2023-02 was already in local storage
File 2023-03 was already in local storage
File 2023-04 was already in local storage
File 2023-05 was already in local storage
File 2023-06 was already in local storage
File 2023-07 was already in local storage
File 2023-08 was already in local storage
File 2023-09 was already in local storage
File 2023-10 was already in local storage
File 2023-11 was already in local storage
File 2023-12 was already in local storage
File 2024-01 was already in local storage
File 2024-02 was already in local storage
2024-03 file is not available
2024-04 file is not available
2024-05 file is not available
2024-06 file is not available
2024-07 file is not available
2024-08 file is not available
2024-09 file is not available
2024-10 file is not available
2024-11 file is not available
2024-12 file is not available


In [4]:
from datetime import datetime
import pandas as pd

from src.data import load_raw_data

from_year = 2022
to_year = datetime.now().year
print(f'Downloading raw data from {from_year} to {to_year}')

rides = pd.DataFrame()

for year in range(from_year, to_year + 1):
  # download data for the whole year
  rides_one_year = load_raw_data(year)

  # append rows
  rides = pd.concat([rides, rides_one_year])

File 2022-01 was already in local storage
File 2022-02 was already in local storage
File 2022-03 was already in local storage
File 2022-04 was already in local storage
File 2022-05 was already in local storage
File 2022-06 was already in local storage
File 2022-07 was already in local storage
File 2022-08 was already in local storage
File 2022-09 was already in local storage
File 2022-10 was already in local storage
File 2022-11 was already in local storage
File 2022-12 was already in local storage
File 2023-01 was already in local storage
File 2023-02 was already in local storage
File 2023-03 was already in local storage
File 2023-04 was already in local storage
File 2023-05 was already in local storage
File 2023-06 was already in local storage
File 2023-07 was already in local storage
File 2023-08 was already in local storage
File 2023-09 was already in local storage
File 2023-10 was already in local storage
File 2023-11 was already in local storage
File 2023-12 was already in local 

In [5]:
print(f'{len(rides)=}')

len(rides)=83936247


In [6]:
from src.data import transform_raw_data_into_ts_data

ts_data = transform_raw_data_into_ts_data(rides)

C:\Users\Victus\Desktop\Pau Lecture\taxi_demand_predictor_service\taxi_demand_predictor\src\data.py:193: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  rides['pickup_hour'] = rides['pickup_datetime'].dt.floor('H')
C:\Users\Victus\Desktop\Pau Lecture\taxi_demand_predictor_service\taxi_demand_predictor\src\data.py:156: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  full_range = pd.date_range(ts_data['pickup_hour'].min(),
100%|██████████| 265/265 [00:06<00:00, 42.76it/s]


In [7]:
import hopsworks

c:\Users\Victus\Desktop\Pau Lecture\taxi_demand_predictor_service\taxi_demand_predictor\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
project = hopsworks.login(
  project=HOPSWORKS_PROJECT_NAME, 
  api_key_value=HOPSWORKS_API_KEY
)

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/655531


In [9]:
feature_store = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.


In [10]:
FEATURE_GROUP_NAME = 'time_series_hourly_feature_group'
FEATURE_GROUP_VERSION = 1

In [11]:
feature_group = feature_store.get_or_create_feature_group(
  name=FEATURE_GROUP_NAME,
  version=FEATURE_GROUP_VERSION,
  description='Time_series data at hourly frequency',
  primary_key = ['pickup_location_id', 'pickup_hour'],
  event_time='pickup_hour'
)

In [12]:
feature_group.insert(ts_data, write_options={'wait_for_job':False})

Uploading Dataframe: 100.00% |██████████| Rows 5024400/5024400 | Elapsed Time: 08:00 | Remaining Time: 00:00


Launching job: time_series_hourly_feature_group_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/655531/jobs/named/time_series_hourly_feature_group_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x1e744f3c310>, None)